# 第12章 计算性能

## 12.1 编译器和解释器

### 练习12.1.1

回顾前几章中感兴趣的模型，能提高它们的计算性能吗？

**解答：**

## 12.2 异步计算

### 练习12.2.1

在CPU上，对本节中相同的矩阵乘法操作进行基准测试，仍然可以通过后端观察异步吗？

**解答：**

## 12.3 自动并行

### 练习12.3.1

在本节定义的`run`函数中执行了八个操作，并且操作之间没有依赖关系。设计一个实验，看看深度学习框架是否会自动地并行地执行它们。

**解答：**

### 练习12.3.2

当单个操作符的工作量足够小，即使在单个CPU或GPU上，并行化也会有所帮助。设计一个实验来验证这一点。

**解答：**

### 练习12.3.3

设计一个实验，在CPU和GPU这两种设备上使用并行计算和通信。

**解答：**

### 练习12.3.4

使用诸如NVIDIA的[Nsight](https://developer.nvidia.com/nsight-compute-2019_5)之类的调试器来验证代码是否有效。

**解答：**

### 练习12.3.5

设计并实验具有更加复杂的数据依赖关系的计算任务，以查看是否可以在提高性能的同时获得正确的结果。

**解答：**

## 12.4 硬件

### 练习12.4.1

编写C语言来测试访问对齐的内存和未对齐的内存之间的速度是否有任何差异。（提示：小心缓存影响。）

**解答：**

### 练习12.4.2

测试按顺序访问或按给定步幅访问内存时的速度差异。

**解答：**

### 练习12.4.3

如何测量CPU上的缓存大小？

**解答：**

### 练习12.4.4

如何在多个内存通道中分配数据以获得最大带宽？如果有许多小的线程，会如何分配？

**解答：**

### 练习12.4.5

一个企业级硬盘正在以10000转/分的速度旋转。在最坏的情况下，硬盘读取数据所需的最短时间是多少（假设磁头几乎是瞬间移动的）？为什么2.5英寸硬盘在商用服务器上越来越流行（相对于3.5英寸硬盘和5.25英寸硬盘）？

**解答：**

### 练习12.4.6

假设HDD制造商将存储密度从每平方英寸1 Tbit增加到每平方英寸5 Tbit。在一个2.5英寸的硬盘上，多少信息能够存储一个环中？内轨和外轨有区别吗？

**解答：**

### 练习12.4.7

从$8$位数据类型到$16$位数据类型，硅片的数量大约增加了四倍，为什么？为什么NVIDIA会在其图灵GPU中添加INT4运算？

**解答：**

### 练习12.4.8

在内存中向前读比向后读快多少？该数字在不同的计算机和CPU供应商之间是否有所不同？为什么？编写C代码进行实验。

**解答：**

### 练习12.4.9

磁盘的缓存大小能否测量？典型的硬盘是多少？固态驱动器需要缓存吗？

**解答：**

### 练习12.4.10

测量通过以太网发送消息时的数据包开销。查找UDP和TCP/IP连接之间的差异。

**解答：**

### 练习12.4.11

直接内存访问允许CPU以外的设备直接向内存写入（和读取）。为什么要这样设计？

**解答：**

### 练习12.4.12

看看Turing T4GPU的性能数字。为什么从FP16到INT8和INT4的性能只翻倍？

**解答：**

### 练习12.4.13

一个网络包从旧金山到阿姆斯特丹的往返旅行需要多长时间？提示：可以假设距离为10000公里。

**解答：**

## 12.5 多GPU训练

### 练习12.5.1

在$k$个GPU上进行训练时，将批量大小从$b$更改为$k \cdot b$，即按GPU的数量进行扩展。

**解答：**

### 练习12.5.2

比较不同学习率时模型的精确度，随着GPU数量的增加学习率应该如何扩展？

**解答：**

### 练习12.5.3

实现一个更高效的`allreduce`函数用于在不同的GPU上聚合不同的参数？为什么这样的效率更高？

**解答：**

### 练习12.5.4

实现模型在多GPU下测试精度的计算。

**解答：**

## 12.6 多GPU的简洁实现

### 练习12.6.1

本节使用ResNet-18，请尝试不同的迭代周期数、批量大小和学习率，以及使用更多的GPU进行计算。如果使用$16$个GPU（例如，在AWS p2.16xlarge实例上）尝试此操作，会发生什么？为什么？

**解答：**

### 练习12.6.2

有时候不同的设备提供了不同的计算能力，我们可以同时使用GPU和CPU，那应该如何分配工作？为什么？

**解答：**

## 12.7 参数服务器

### 练习12.7.1

请尝试进一步提高环同步的性能吗。（提示：可以双向发送消息。）

**解答：**

### 练习12.7.2

在计算仍在进行中，可否允许执行异步通信？它将如何影响性能？

**解答：**

### 练习12.7.3

怎样处理在长时间运行的计算过程中丢失了一台服务器这种问题？尝试设计一种容错机制来避免重启计算？

**解答：**